In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Veri setini yükleme
df = pd.read_excel("/kaggle/input/co2emisyon/dataset.xlsx", 0)
def change(col):
    marka_kodlari = {
        'Acura': 0,
        'Audi': 1,
        'BMW': 2,
        'Buick': 3,
        'Chevrolet': 4,
        'Dodge': 5,
        'Ford': 6,
        'GMC': 7,
        'Honda': 8,
        'Hyundai': 9,
        'Jaguar': 10,
        'Jeep': 11,
        'Land Rover': 12,
        'Mazda': 13,
        'Mercedes-BenG': 14,
        'Mitsubishi': 15,
        'Nissan': 16,
        'Plymouth': 17,
        'Toyota': 18,
        'Volkswagen': 19
    }
    return marka_kodlari.get(col, 20)  # 20, listelenmeyen markalar için

df['Make'] = df['Make'].apply(change)
print(df.head())
# Veriyi hazırlama
X = df[['Model_Year', 'Make', 'Engine_Size', 'Combined', 'Fuel_Type']].values
y = df['CO2Emissions'].values


In [ ]:

df = df.drop('Model_Year', axis=1)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def format_column_name(column):
   # CO2Emissions gibi özel durumları kontrol et
   if 'CO2' in column:
       return 'CO2 emission'

   # Diğer sütun isimleri için
   words = column.split('_')
   # İlk kelimenin ilk harfi büyük, diğerleri küçük
   formatted_words = [words[0].capitalize()]
   # Sonraki kelimeler küçük harfle başlasın
   formatted_words.extend(word.lower() for word in words[1:])
   return ' '.join(formatted_words)

def plot_correlations(df, target_column):
   # Sütun isimlerini düzenle
   df = df.copy()
   df.columns = [format_column_name(col) for col in df.columns]

   # Hedef sütun ismini formatla
   target_column = format_column_name(target_column)

   # Korelasyonları hesapla ve hedef sütuna göre sırala
   correlations = df.corr()[target_column].sort_values(ascending=False)

   # Figür boyutunu ayarla
   plt.figure(figsize=(12, 6))

   # Çubuk grafiği oluştur
   bars = plt.barh(range(len(correlations)), correlations)

   # Çubukları renklendir
   for i, bar in enumerate(bars):
       if correlations[i] >= 0:
           bar.set_color(plt.cm.RdBu(0.75 + correlations[i]/4))
       else:
           bar.set_color(plt.cm.RdBu(0.25 + correlations[i]/4))

   # Eksenleri ayarla
   plt.yticks(range(len(correlations)), correlations.index, fontsize=10)
   plt.xlabel('Correlation Coefficient', fontsize=12)
   plt.title(f'Correlation between {target_column} and Other Variables', fontsize=14)

   # Izgara ekle
   plt.grid(True, alpha=0.3)

   # Değerleri çubukların üzerine yaz
   for i, v in enumerate(correlations):
       plt.text(v + 0.01 if v >= 0 else v - 0.05,
               i,
               f'{v:.2f}',
               va='center',
               fontsize=10)

   # Görsel düzenlemeler
   plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
   plt.xlim(-1, 1)

   # Grafiği göster
   plt.tight_layout()
   plt.show()

   # Korelasyon matrisi
   plt.figure(figsize=(10, 8),dpi=100)
   sns.heatmap(df.corr(),
               annot=True,
               cmap='RdBu_r',
               center=0,
               fmt='.2f',
               square=True,
               vmin=-1,
               vmax=1,
               linewidths=2,
               linecolor='white',
               annot_kws={'size': 15},
               cbar_kws={'orientation': 'vertical',
                        'pad': 0.05,
                        'aspect': 25})

   plt.title('Correlation graph between all variables', fontsize=20)
   plt.xticks(rotation=45, ha='right', fontsize=15)
   plt.yticks(fontsize=15)
   plt.tight_layout()
   plt.show()

# Kullanım:
# plot_correlations(df, 'engine_size')  # Engine size olarak görünecek
plot_correlations(df, 'CO2Emissions')  # CO2 emission olarak görünecek

Yeni

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM,SimpleRNN
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time
from datetime import datetime
from sklearn.metrics import explained_variance_score


# Time formatting function
def format_time(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    return f"{int(minutes)}min {seconds:.2f}sec"

# Model summary printing function
def print_model_summary(model):
    print("\nModel Architecture:")
    print("=" * 50)
    model.summary()

    # Print model parameters
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    non_trainable_params = total_params - trainable_params

    print("\nModel Parameters:")
    print("=" * 50)
    print(f"Total number of parameters: {total_params:,}")
    print(f"Number of trainable parameters: {trainable_params:,}")
    print(f"Number of non-trainable parameters: {non_trainable_params:,}")

# Save start time
start_time = time.time()

# Load data (You'll need to adjust this part according to your dataset)
# Example: data = pd.read_csv('your_data.csv')
# X = data.drop('target_column', axis=1)
# y = data['target_column']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

# Reshape data for LSTM
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Calculate data preparation time
data_prep_time = time.time() - start_time

# Model hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 32
EPOCHS = 100
DROPOUT_RATE = 0.3
L2_LAMBDA = 0.01
RNN_UNITS_1 = 64
RNN_UNITS_2 = 32
DENSE_UNITS = 16

print("\nModel Input Shape:")
print("=" * 50)
print(f"Input dimensions: {X_train_reshaped.shape}")
print(f"Number of features: {X_train.shape[1]}")
print(f"Number of time steps: 1")
print(f"Batch size: {BATCH_SIZE}")



# RNN model architecture
model = Sequential([
    SimpleRNN(RNN_UNITS_1, activation='relu', input_shape=(1, X_train.shape[1]),
         return_sequences=True, name='rnn_1'),
    Dropout(DROPOUT_RATE, name='dropout_1'),
    SimpleRNN(RNN_UNITS_2, activation='relu', name='rnn_2'),
    Dropout(DROPOUT_RATE, name='dropout_2'),
    Dense(DENSE_UNITS, activation='relu',
          kernel_regularizer=tf.keras.regularizers.l2(L2_LAMBDA), name='dense_1'),
    Dense(1, name='output')
])



# Model summary
print_model_summary(model)

# Optimizer
optimizer = Adam(learning_rate=LEARNING_RATE)

# Compile model
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=10,
    min_lr=0.00001,
    verbose=1
)

# Training start time
train_start_time = time.time()

# Model training
history = model.fit(
    X_train_reshaped, y_train_scaled,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    callbacks=[ reduce_lr],
    verbose=1
)

# Calculate training time
training_time = time.time() - train_start_time

# Test start time
test_start_time = time.time()

# Make predictions on test set
y_pred_train_scaled = model.predict(X_train_reshaped).flatten()
y_pred_scaled = model.predict(X_test_reshaped).flatten()

# Calculate testing time
testing_time = time.time() - test_start_time

# Normalized metrics
# Normalized metrics
r2_train_norm = r2_score(y_train_scaled, y_pred_train_scaled)
mse_train_norm = mean_squared_error(y_train_scaled, y_pred_train_scaled)
rmse_train_norm = np.sqrt(mse_train_norm)
mae_train_norm = np.mean(np.abs(y_train_scaled - y_pred_train_scaled))

# Mean Absolute Percentage Error (MAPE)
mape_train_norm = np.mean(np.abs((y_train_scaled - y_pred_train_scaled) / y_train_scaled)) * 100

# Adjusted R² (Düzeltilmiş R²) for training
n_train = len(y_train_scaled)  # Number of training samples
p = X_train_reshaped.shape[1]  # Number of features
adjusted_r2_train_norm = 1 - ((1 - r2_train_norm) * (n_train - 1) / (n_train - p - 1))

# Explained Variance Score
explained_variance_train_norm = explained_variance_score(y_train_scaled, y_pred_train_scaled)

r2_norm = r2_score(y_test_scaled, y_pred_scaled)
mse_norm = mean_squared_error(y_test_scaled, y_pred_scaled)
rmse_norm = np.sqrt(mse_norm)
mae_norm = np.mean(np.abs(y_test_scaled - y_pred_scaled))

# Mean Absolute Percentage Error (MAPE)
mape_norm = np.mean(np.abs((y_test_scaled - y_pred_scaled) / y_test_scaled)) * 100

# Adjusted R² (Düzeltilmiş R²) for test
n_test = len(y_test_scaled)  # Number of test samples
adjusted_r2_norm = 1 - ((1 - r2_norm) * (n_test - 1) / (n_test - p - 1))

# Explained Variance Score
explained_variance_norm = explained_variance_score(y_test_scaled, y_pred_scaled)

# Original scale metrics
y_test_original = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()
y_pred_original = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

r2_orig = r2_score(y_test_original, y_pred_original)
mse_orig = mean_squared_error(y_test_original, y_pred_original)
rmse_orig = np.sqrt(mse_orig)
mae_orig = np.mean(np.abs(y_test_original - y_pred_original))

# Mean Absolute Percentage Error (MAPE) for original scale
mape_orig = np.mean(np.abs((y_test_original - y_pred_original) / y_test_original)) * 100

# Adjusted R² (Düzeltilmiş R²) for original scale
adjusted_r2_orig = 1 - ((1 - r2_orig) * (n_test - 1) / (n_test - p - 1))

# Explained Variance Score for original scale
explained_variance_orig = explained_variance_score(y_test_original, y_pred_original)

# Print the metrics
print(f"Normalized Metrics (Training):")
print(f"R²: {r2_train_norm}")
print(f"MSE: {mse_train_norm}")
print(f"RMSE: {rmse_train_norm}")
print(f"MAE: {mae_train_norm}")
print(f"MAPE: {mape_train_norm}")
print(f"Adjusted R²: {adjusted_r2_train_norm}")
print(f"Explained Variance Score: {explained_variance_train_norm}")

print(f"\nNormalized Metrics (Test):")
print(f"R²: {r2_norm}")
print(f"MSE: {mse_norm}")
print(f"RMSE: {rmse_norm}")
print(f"MAE: {mae_norm}")
print(f"MAPE: {mape_norm}")
print(f"Adjusted R²: {adjusted_r2_norm}")
print(f"Explained Variance Score: {explained_variance_norm}")

print(f"\nOriginal Scale Metrics (Test):")
print(f"R²: {r2_orig}")
print(f"MSE: {mse_orig}")
print(f"RMSE: {rmse_orig}")
print(f"MAE: {mae_orig}")
print(f"MAPE: {mape_orig}")
print(f"Adjusted R²: {adjusted_r2_orig}")
print(f"Explained Variance Score: {explained_variance_orig}")

# Evaluation time
# Print results
print("\nTime Metrics:")
print("=" * 50)
print(f"Data preparation time: {format_time(data_prep_time)}")
print(f"Training time: {format_time(training_time)}")
print(f"Testing time: {format_time(testing_time)}")
print(f"Total time: {format_time(data_prep_time + training_time + testing_time)}")

print("\nModel Hyperparameters:")
print("=" * 50)
print(f"Learning rate: {LEARNING_RATE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Maximum epochs: {EPOCHS}")
print(f"Actual epochs: {len(history.history['loss'])}")
print(f"Dropout rate: {DROPOUT_RATE}")
print(f"L2 regularization lambda: {L2_LAMBDA}")
print(f"LSTM layer 1 units: {LSTM_UNITS_1}")
print(f"LSTM layer 2 units: {LSTM_UNITS_2}")
print(f"Dense layer units: {DENSE_UNITS}")

print("\nModel Performance Metrics:")
print("=" * 50)
print(f"                  Normalized      Original")
print(f"R-squared (R²):   {r2_norm:.4f}        {r2_orig:.4f}")
print(f"MSE:              {mse_norm:.4f}        {mse_orig:.4f}")
print(f"RMSE:             {rmse_norm:.4f}        {rmse_orig:.4f}")
print(f"MAE:              {mae_norm:.4f}        {mae_orig:.4f}")

# Visualize training process
plt.figure(figsize=(15, 5))

# Loss plot
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

# Learning rate change
plt.subplot(1, 3, 2)
lr_values = [tf.keras.backend.get_value(model.optimizer.learning_rate)]
for i in range(1, len(history.history['loss'])):
    if 'lr' in history.history:
        lr_values.append(history.history['lr'][i-1])
    else:
        lr_values.append(lr_values[-1])
plt.plot(lr_values)
plt.title('Learning Rate Change')
plt.ylabel('Learning Rate')
plt.xlabel('Epoch')

# Actual vs Predicted
plt.subplot(1, 3, 3)
plt.scatter(y_test_original, y_pred_original, alpha=0.5)
plt.plot([y_test_original.min(), y_test_original.max()],
         [y_test_original.min(), y_test_original.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values')

plt.tight_layout()
plt.show()


#++++++++++++++++++++++++++++



import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Hata hesaplama
errors = y_test_original - y_pred_original

# Grafik estetik ayarları
plt.figure(figsize=(12, 8))
sns.set_style("whitegrid")  # Modern stil
sns.set_palette("pastel")  # Yumuşak renk paleti

# Histogram ve KDE çizme
sns.histplot(errors, kde=False, color="#6495ED", edgecolor="black", alpha=0.7, label="Histogram")
sns.kdeplot(errors, color="#2E8B57", linewidth=3, label="KDE (Density)")

# Başlık ve eksen etiketleri
plt.title('RNN Prediction Error Distribution', fontsize=20, weight='bold', color="black", pad=20)
plt.xlabel('Prediction Error', fontsize=16, weight='bold', color="black")
plt.ylabel('Frequency', fontsize=16, weight='bold', color="black")

# X ve Y eksenlerini genişletme
plt.xlim(min(errors) - 5, max(errors) + 5)
plt.ylim(0, plt.gca().get_ylim()[1] * 1.2)  # Dikey ekseni genişletme

# Eksen yazı büyüklükleri
plt.xticks(fontsize=14, color="black", weight='bold')
plt.yticks(fontsize=14, color="black", weight='bold')

# Legend ekleme
plt.legend(fontsize=14, loc='upper right', frameon=True, shadow=True, facecolor='white')

# Düzeni optimize etme
plt.tight_layout()

# Grafiği gösterme
plt.show()


#+++++++++++++++++++



# Error distribution histogram
errors = y_test_original - y_pred_original
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")  # Modern stil
sns.set_palette("pastel")  # Yumuşak renk paleti

sns.histplot(errors, kde=True)
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.xticks(fontsize=14, color="black", weight='bold')
plt.yticks(fontsize=14, color="black", weight='bold')
plt.title('RNN Prediction Error Distribution', fontsize=20, weight='bold', color="black", pad=20)
plt.tight_layout()
plt.show()


#++++++++++++++++++++++






# Save model
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model.save(f'lstm_model_{timestamp}.h5')
print(f"\nModel saved: lstm_model_{timestamp}.h5")

# Make prediction for new data
new_data = np.array([[2025, 3, 1.6, 8.0, 3]])  # Example new data
new_data_scaled = scaler_X.transform(new_data)
new_data_reshaped = new_data_scaled.reshape((1, 1, new_data_scaled.shape[1]))
prediction_scaled = model.predict(new_data_reshaped)
prediction_original = scaler_y.inverse_transform(prediction_scaled)
print(f"\nPrediction for New Data: {prediction_original[0][0]:.2f}")

# Clear memory
tf.keras.backend.clear_session()

In [ ]:
 # Error distribution histogram
errors = y_test_original - y_pred_original
plt.figure(figsize=(10, 6),dpi=100)
sns.set_style("whitegrid")  # Modern stil
sns.set_palette("pastel")  # Yumuşak renk paleti

mean_error = np.mean(errors)
median_error = np.median(errors)
std_dev = np.std(errors)

plt.plot( '*', markersize=6,
         label='KDE (Density)', color='blue')

plt.axvline(mean_error, color='green', linestyle='dashed', linewidth=2, label=f'Mean: {mean_error:.2f}')
plt.axvline(median_error, color='yellow', linestyle='dashed', linewidth=2, label=f'Median: {median_error:.2f}')
plt.axvspan(mean_error - std_dev, mean_error + std_dev, color='green', alpha=0.1, label='1 Std Dev')


sns.histplot(errors, kde=True, color="blue", edgecolor="red", alpha=0.3, label="Histogram")
plt.xlabel('Prediction Error', fontsize=20, weight='bold', color="black")
plt.ylabel('Frequency', fontsize=20, weight='bold', color="black")
plt.title('Distribution of Prediction Errors')
plt.xticks(fontsize=16, color="black", weight='bold')
plt.yticks(fontsize=16, color="black", weight='bold')
plt.title('RNN Prediction Error Distribution', fontsize=20, weight='bold', color="black", pad=20)
plt.legend(fontsize=16, loc='upper left', frameon=True, shadow=True, facecolor='white')
plt.tight_layout()
plt.show()


In [ ]:
# Continue from previous code...

# New performance plot
plt.figure(figsize=(8, 6),dpi=100)

# Calculate and add R-squared value
r2_value = r2_score(y_test_original, y_pred_original)
plt.plot([], [], ' ', label=f'R$^2$: {r2_value:.2f}')

# Scatter plot of predictions vs actual values
plt.plot(y_test_original, y_pred_original, '*', markersize=6,
         label='Actual vs Predicted Values', color='red')

# Regression line
plt.plot([y_test_original.min(), y_test_original.max()],
         [y_test_original.min(), y_test_original.max()],
         'r--', label='Regression Line', color='blue', linewidth=4)

# Graph titles and labels
plt.xlabel('Actual Values', fontsize=20)
plt.ylabel('Predicted Values', fontsize=20)
plt.title('RNN Regression Plot', fontsize=20)

# Legend settings
legend = plt.legend(fontsize=14, loc='upper left',
                   facecolor='#D7F8FD', frameon=True)
legend.get_frame().set_alpha(0.8)

# Grid and other visual settings
plt.grid(True)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.tight_layout()
plt.show()